In [1]:
import re


def get_verdict(markdown_content, provision):
    verdicts = []

    target = rf"\s{provision.strip()}[\s]+$"
    start_section = r"Assignment of verdict[\s]+$"
    middle_section = r"^The verdict FAIL is assigned otherwise.[\s]+$"
    end_section = r"^##[\w\s.-]+$"

    # 使用正則表達式搜索目標字符串
    pattern = re.compile(target, re.IGNORECASE | re.MULTILINE)
    match = pattern.search(markdown_content)

    if match:
        # print(match)
        # 找到目標字符串後，從該位置開始搜索最近的start_section和end_section
        start_pos = match.end()
        start_section_pattern = re.compile(start_section, re.IGNORECASE | re.MULTILINE)
        middle_section_pattern = re.compile(
            middle_section, re.IGNORECASE | re.MULTILINE
        )
        end_section_pattern = re.compile(end_section, re.IGNORECASE | re.MULTILINE)

        end_section_match = end_section_pattern.search(markdown_content, start_pos)
        # print(end_section_match)
        if end_section_match:
            end_pos = end_section_match.start()
        else:
            end_pos = len(markdown_content)
            print("No end_section found, using end of content")

        current_pos = start_pos
        while current_pos < end_pos:
            start_section_match = start_section_pattern.search(
                markdown_content, current_pos
            )
            if not start_section_match or start_section_match.start() >= end_pos:
                break

            middle_section_match = middle_section_pattern.search(
                markdown_content, start_section_match.end()
            )
            if middle_section_match and middle_section_match.start() < end_pos:
                extracted_content = markdown_content[
                    start_section_match.end() : middle_section_match.end()
                ].strip()
                verdicts.append(extracted_content)
                current_pos = middle_section_match.end()
            else:
                break
    else:
        print("Target not found")

    if len(verdicts) > 1:
        verdict = "The verdict PASS is assigned if:  \n"
        for item in verdicts:
            splits = re.split(r"[\n]", item)
            for split in splits:
                if split.startswith("-"):
                    verdict += f"{split}\n"
        verdict += "The verdict FAIL is assigned otherwise."
    elif len(verdicts) == 1:
        verdict = verdicts[0]
    else:
        verdict = "Not found"

    return verdict

In [2]:
import pandas as pd

df = pd.read_csv("../docs/EN303645問題彙整(verdict)_台科大.csv")

In [3]:
markdown_content = None
with open("../docs/ts_103701_only_test_scenario.md", "r") as f:
    markdown_content = f.read()

terminate = get_verdict(markdown_content, "5.3-3")
terminate

'The verdict PASS is assigned if:\n\n- each software component is covered by at least one update mechanism, which is simple for the user to apply.\n\nThe verdict FAIL is assigned otherwise.'

In [4]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

verdict_list = []
for idx in range(len(df)):
    # create a query
    row = df.iloc[idx]
    verdict_list.append(get_verdict(markdown_content, row["provision"].strip()))

final_df = df.iloc[:, [0]].copy()
final_df["criteria"] = verdict_list
final_df

No end_section found, using end of content


,provision,criteria
0,5.1-1,"The verdict PASS is assigned if: \n- each password of a password-based authentication mechanism being used in any state other than the factory default, that is not defined by the user, is unique per device.\n- every discovered password-based authentication mechanism is documented in the IXIT; and\n- the user is required to define all passwords before being used, that are stated as defined by the user in the IXIT;\n- there is no indication that the generation of a not user-defined password of the DUT used in any state other than the factory default differs from the generation mechanism described in the IXIT.\nThe verdict FAIL is assigned otherwise."
1,5.1-2,"The verdict PASS is assigned if: \n- no obvious regularities in pre-installed passwords are found; and\n- no common strings or other common patterns in pre-installed passwords are found; and\n- the generation mechanisms for pre-installed passwords do not induce passwords, that are related in an obvious way to public information; and\n- the generation mechanisms for pre-installed passwords are considered appropriate in terms of complexity.\n- for each pre-installed password there is no indication, that its generation differs from the generation mechanism described in the IXIT.\nThe verdict FAIL is assigned otherwise."
2,5.1-3,The verdict PASS is assigned if: \n- the security guarantees are appropriate for the use case of user authentication; and\n- the mechanism is appropriate to achieve the security guarantees with respect to the use case; and\n- all used cryptographic details are considered as best practice for the use case; and\n- all used cryptographic details are not known to be vulnerable to a feasible attack for the desired security property.\n- there is no indication that any used cryptographic setting differs from its IXIT documentation.\nThe verdict FAIL is assigned otherwise.
3,5.1-4,The verdict PASS is assigned if: \n- for all user based authentication mechanisms the published resource describes how to change the authentication value with a simple mechanism.\n- all mechanisms for the user to change authentication values for user authentication mechanisms work as described.\nThe verdict FAIL is assigned otherwise.
4,5.1-5,The verdict PASS is assigned if: \n- the documented mechanisms make brute force attacks via network interfaces impracticable.\n- ever discovered network-based authentication mechanism is documented in the IXIT; and\n- for all authentication mechanism via network interfaces there is no indication that the implementation of brute force prevention differs from its IXIT documentation.\nThe verdict FAIL is assigned otherwise.
5,5.2-1,The verdict PASS is assigned if: \n- the publication of the vulnerability disclosure policy is available for anybody.\n- the vulnerability disclosure policy is publicly accessible; and\n- the vulnerability disclosure policy contains contact information and information on timelines regarding acknowledgement of receipt and status updates.\nThe verdict FAIL is assigned otherwise.
6,5.2-2,Assignment of verdict The verdict PASS is assigned if:\n\n- there is no indication that any described kind of vulnerability is not acted on timely; and\n- a confirmation for the implementation is given.\n\nThe verdict FAIL is assigned otherwise.
7,5.2-3,The verdict PASS is assigned if:\n\n- the described way is suited for continuously monitoring for security vulnerabilities; and\n- the described way is suited for identifying security vulnerabilities; and\n- the described way is suited for rectifying security vulnerabilities; and\n- a confirmation for the implementation is given.\n\nThe verdict FAIL is assigned otherwise.
8,5.3-1,"The verdict PASS is assigned if:\n\n- for all software components without the ability for software updates, a software update is not possible for practicability reasons or security reasons; and\n- no update mechanism can be misused by an attacker.\n\nThe verdict FAIL is assigned otherwis

In [5]:
not_found_df = final_df[final_df['criteria'] == 'Not found']
not_found_df

,provision,criteria


In [6]:
df = df.iloc[:, :-1]
df['criteria'] = verdict_list
df.to_csv("../docs/test_search_verdicts_with_regex.csv", index=False, encoding="utf-8-sig")